<a href="https://colab.research.google.com/github/junwei2110/FreeCodeCamp_MachineLearning/blob/SMS-text-classifier/FCC_SMS_Challenge_Using_characters.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import keras
import tensorflow as tf
from tensorflow import keras
import pandas as pd
from keras.preprocessing import sequence
import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
!wget https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv

train_file_path = "train-data.tsv"
test_file_path = "valid-data.tsv"

--2021-07-10 10:50:58--  https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 172.67.70.149, 104.26.2.33, 104.26.3.33, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|172.67.70.149|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 358233 (350K) [text/tab-separated-values]
Saving to: ‘train-data.tsv’

train-data.tsv      100%[===================>] 349.84K  --.-KB/s    in 0.005s  

2021-07-10 10:50:58 (65.4 MB/s) - ‘train-data.tsv’ saved [358233/358233]

--2021-07-10 10:50:58--  https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 172.67.70.149, 104.26.2.33, 104.26.3.33, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|172.67.70.149|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 118774 (116K) [text/tab-separated-values]
Saving to: ‘valid-data.tsv’

valid-data.tsv      100%[==========

In [ ]:
# Read the data
training_raw = open(train_file_path, 'rb').read().decode(encoding='utf-8')
testing_raw = open(test_file_path, 'rb').read().decode(encoding='utf-8') 
print(training_raw[:250])

ham	ahhhh...just woken up!had a bad dream about u tho,so i dont like u right now :) i didnt know anything about comedy night but i guess im up for it.
ham	you can never do nothing
ham	now u sound like manky scouse boy steve,like! i is travelling on d


In [ ]:
# Create a list with the labels and the corresponding text
import re

training_text = re.split('[\t\n]', training_raw)
testing_text = re.split('[\t\n]', testing_raw)

In [ ]:
# Split into the text and the labels

training_text_data = []
training_text_label = []
testing_text_data = []
testing_text_label = []

for i in range(len(training_text)):
  if (i%2 == 0):
    training_text_label.append(training_text[i])
  else:
    training_text_data.append(training_text[i])

for j in range(len(testing_text)):
  if (j%2 == 0):
    testing_text_label.append(testing_text[j])
  else:
    testing_text_data.append(testing_text[j])

In [ ]:
# Check the len of the text and labels

if len(training_text_data) < len(training_text_label):
  training_text_label.pop(-1)
elif len(training_text_data) > len(training_text_label):
  training_text_data.pop(-1)

if len(testing_text_data) < len(testing_text_label):
  testing_text_label.pop(-1)
elif len(testing_text_data) > len(testing_text_label):
  testing_text_data.pop(-1)

In [ ]:
# Our next objective is to create dictionary of character-num pairs. We need to convert the characters into numbers to fit into the model, and decode the numbers back into characters for interpretation
# Reason why words are not chosen is because words are not exhaustive, and in this case, there may be a lot of new words in the testing set that does not appear in the training data

# We make the raw training data into a set to iron out all the unique characters and then sort them back into a list
char_unique = sorted(set(training_raw))

# Now we create the dictionary using the awesome enumerate method which converts the characters here into tupples paired with the index of the characters
char2idx = {u:i for i,u in enumerate(char_unique)}

print(char2idx)

{'\t': 0, '\n': 1, ' ': 2, '!': 3, '#': 4, '$': 5, '%': 6, '&': 7, "'": 8, '(': 9, ')': 10, '*': 11, '+': 12, ',': 13, '-': 14, '.': 15, '/': 16, '0': 17, '1': 18, '2': 19, '3': 20, '4': 21, '5': 22, '6': 23, '7': 24, '8': 25, '9': 26, ':': 27, ';': 28, '<': 29, '=': 30, '>': 31, '?': 32, '@': 33, '[': 34, '\\': 35, ']': 36, '^': 37, '_': 38, 'a': 39, 'b': 40, 'c': 41, 'd': 42, 'e': 43, 'f': 44, 'g': 45, 'h': 46, 'i': 47, 'j': 48, 'k': 49, 'l': 50, 'm': 51, 'n': 52, 'o': 53, 'p': 54, 'q': 55, 'r': 56, 's': 57, 't': 58, 'u': 59, 'v': 60, 'w': 61, 'x': 62, 'y': 63, 'z': 64, '|': 65, '~': 66, '\x91': 67, '\x92': 68, '\x93': 69, '\x96': 70, '£': 71, '»': 72, 'è': 73, 'ú': 74, 'ü': 75, '–': 76, '‘': 77, '’': 78, '“': 79, '…': 80}


In [ ]:
# We check what is the longest character length for the training data

maxlen = 0
for i in range(len(training_text_data)):
  if maxlen <= len(training_text_data[i]):
    maxlen = len(training_text_data[i])

maxlen

910

In [ ]:
# Now we create the encoder to convert text to int:

def text_to_int(text):
  list_of_int = []
  for c in text:
    if c in char2idx:
      list_of_int.append(char2idx[c])      
    else:
      list_of_int.append(0)

  list_of_int = np.array(list_of_int)  
  # Now we need to stretch the length to ensure that all the strings evaluated are of same character length
  return sequence.pad_sequences([list_of_int], maxlen)[0] 


# We also create the decoder to convert int back to text:
idx2char = np.array(char_unique)

def int_to_text(ints):
  return "".join(idx2char[ints])

In [ ]:
# To test the above

print(training_raw[:20])
text_as_int = text_to_int(training_raw[:20])
print(text_as_int)
print(int_to_text(text_as_int))

ham	ahhhh...just wok
[ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0

In [ ]:
# The next step will be to convert the training_text_data and testing_text_data into integer form

for i in range(len(training_text_data)):
  training_text_data[i] = text_to_int(training_text_data[i])

for j in range(len(testing_text_data)):
  testing_text_data[j] = text_to_int(testing_text_data[j])


In [ ]:
# Next we convert the labels into integer form as well

labels_dict = {"ham" : 0, "spam": 1}
labels_decoder = {0: "ham", 1: "spam"}

def label_to_int(label):
  return labels_dict[label]

def int_to_label(ints):
  return "".join(labels_decoder[ints])

for i in range(len(training_text_label)):
  training_text_label[i] = label_to_int(training_text_label[i])

for j in range(len(testing_text_label)):
  testing_text_label[j] = label_to_int(testing_text_label[j])

In [ ]:
# Finally time to create the model
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(len(char_unique),256),
    tf.keras.layers.LSTM(32),
    tf.keras.layers.Dense(1, activation = 'sigmoid')    #Sigmoid ensures output is between 0 and 1
])

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, None, 256)         20736     
_________________________________________________________________
lstm (LSTM)                  (None, 32)                36992     
_________________________________________________________________
dense (Dense)                (None, 1)                 33        
Total params: 57,761
Trainable params: 57,761
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model.compile(loss = 'binary_crossentropy', optimizer = 'rmsprop', metrics = ['acc'])

history = model.fit(np.array(training_text_data), np.array(training_text_label), epochs = 5)

Epoch 1/5
131/131 [==============================] - 7s 38ms/step - loss: 0.0356 - acc: 0.9904
Epoch 2/5
131/131 [==============================] - 5s 38ms/step - loss: 0.0328 - acc: 0.9909
Epoch 3/5
131/131 [==============================] - 5s 38ms/step - loss: 0.0297 - acc: 0.9916
Epoch 4/5
131/131 [==============================] - 5s 39ms/step - loss: 0.0283 - acc: 0.9921
Epoch 5/5
131/131 [==============================] - 5s 38ms/step - loss: 0.0281 - acc: 0.9916


In [ ]:
results = model.evaluate(np.array(testing_text_data), np.array(testing_text_label))

print(results)

44/44 [==============================] - 1s 22ms/step - loss: 0.0661 - acc: 0.9878
[0.06608466058969498, 0.9877873659133911]


In [ ]:
# function to predict messages based on model
# (should return list containing prediction and label, ex. [0.008318834938108921, 'ham'])
def predict_message(pred_text):

  pred_text2int = text_to_int(pred_text)
  pred = np.zeros((1,maxlen))
  pred[0] = pred_text2int
  result = model.predict(pred)

  prediction = list(result[0])
  if result < 0.5:
    prediction.append("ham")
  else:
    prediction.append("spam")

  return (prediction)

pred_text = "how are you doing today?"

prediction = predict_message(pred_text)
print(prediction)

[0.000106733765, 'ham']


In [ ]:
# Run this cell to test your function and model. Do not modify contents.
def test_predictions():
  test_messages = ["how are you doing today",
                   "sale today! to stop texts call 98912460324",
                   "i dont want to go. can we try it a different day? available sat",
                   "our new mobile video service is live. just install on your phone to start watching.",
                   "you have won £1000 cash! call to claim your prize.",
                   "i'll bring it tomorrow. don't forget the milk.",
                   "wow, is your arm alright. that happened to me one time too"
                  ]

  test_answers = ["ham", "spam", "ham", "spam", "spam", "ham", "ham"]
  passed = True
  list_of_pred = []
  for msg, ans in zip(test_messages, test_answers):
    prediction = predict_message(msg)
    list_of_pred.append(prediction)
    if prediction[1] != ans:
      passed = False

  print(list_of_pred)

  if passed:
    print("You passed the challenge. Great job!")
  else:
    print("You haven't passed yet. Keep trying.")

test_predictions()

[[0.00034755946, 'ham'], [0.99918276, 'spam'], [0.0014556174, 'ham'], [0.00061522675, 'ham'], [0.9927176, 'spam'], [5.1708244e-05, 'ham'], [0.0011902421, 'ham']]
You haven't passed yet. Keep trying.
